# Article Content Extraction 


In [9]:
# Get current timestamp.
from datetime import *
now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M:%S.000")

In [10]:
# Runtime configuration.
NUM_ARTICLES = 10
SOURCE_CSV = '2020.csv'
OUTPUT_CSV = 'UpennBox_SL_2020_content_extracted2.csv'
ENV = 'local' # runtime environment (local, colab)
BATCH_ROWS = 100 # number of rows per batch
BATCH_DELAY = 0 # number of seconds between batches
BEFORE_DATE = now # YYYY-MM-DD HH:MM:SS.000
AFTER_DATE = '2019-01-01 00:00:00.000' # YYYY-MM-DD HH:MM:SS.000

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
# This will run if goose3 is not already installed.
%pip install goose3

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Load input into a dataframe.
import pandas as pd
df = pd.read_csv(SOURCE_CSV)
df.head()

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,locations,organizations,themes,persons,tone,dates,amounts,translationinfo,country,year
0,20200316171500-563,2020-03-16 17:15:00.000,1,570news.com,https://www.570news.com/2020/03/16/new-africa-...,"[{count_type=KILL, count=11, object_type=, loc...","[{location_type=4, location_fullname=Monrovia,...","[{organization=Associated Press, character_off...","[{theme=UNGP_EDUCATION, character_offset=4791}...","[{person=Shakira Choonara, character_offset=40...","{tone=-3.86052303860523, positive_score=3.1133...",NaN,"[{amount=30, object=of Africa 54 countries, ch...",NaN,SL,2020
1,20200316171500-973,2020-03-16 17:15:00.000,1,yahoo.com,https://news.yahoo.com/africell-holding-comple...,NaN,"[{location_type=1, location_fullname=Sierra Le...","[{organization=Africell Holding, character_off...","[{theme=GENERAL_GOVERNMENT, character_offset=1...","[{person=Ziad Dalloul, character_offset=817}, ...","{tone=4.67445742904841, positive_score=5.34223...",NaN,"[{amount=12000000, object=customers, character...",NaN,SL,2020
2,20200316171500-1417,2020-03-16 17:15:00.000,1,stuff.co.nz,https://www.stuff.co.nz/timaru-herald/news/120...,NaN,"[{location_type=1, location_fullname=Sierra Le...",NaN,"[{theme=TAX_WORLDFISH_BANJOS, character_offset...","[{person=John Trotter, character_offset=2650},...","{tone=1.6, positive_score=2.2, negative_score=...","[{date_resolution=1, month=0, day=0, year=1963...",NaN,NaN,SL,2020
3,20200316171500-1483,2020-03-16 17:15:00.000,1,bmj.com,https://www.bmj.com/content/368/bmj.m481/rr,NaN,"[{location_type=1, location_fullname=Sierra Le...",NaN,"[{theme=GENERAL_GOVERNMENT, character_offset=3...","[{person=Jenny Gibson, character_offset=63}]","{tone=1.36054421768708, positive_score=3.40136...",NaN,NaN,NaN,SL,2020
4,20200316171500-2485,2020-03-16 17:15:00.000,1,venturesafrica.com,http://venturesafrica.com/apostories/25-police...,NaN,"[{location_type=1, location_fullname=Sierra Le...",[{organization=Police Contributing Countries P...,"[{theme=TAX_FNCACT_ASSISTANT, character_offset...","[{person=Rex Dundun, character_offset=1319}, {...","{tone=1.04712041884817, positive_score=2.35602...",NaN,"[{amount=25, object=newly deployed Individual ...",NaN,SL,2020


In [20]:
# Restrict by date.
period_df = df[(df['date'] < BEFORE_DATE) & (df['date'] > AFTER_DATE)]
period_df.shape

(36000, 16)

In [27]:
# Restrict by number of rows.
if len(period_df) > NUM_ARTICLES:
    small_df = period_df[:NUM_ARTICLES]
else:
    small_df = period_df

small_df.shape

true


(10, 16)

In [28]:
# Run dataframe rows through Goose3 extraction.
from goose3 import Goose
import csv
import time

g = Goose()
small_df_copy = period_df[['gkgrecordid', 'date', 'documentidentifier']]
small_df_copy['content'] = ''
batch = 0

for index, row in small_df.iterrows():
    
    batch = batch + 1
    if batch >= BATCH_ROWS:
        time.sleep(BATCH_DELAY)
        batch = 0

    url = row['documentidentifier']
    
    try:
        g.extract(url=url)
        article = g.extract(url=url)
        content = article.cleaned_text
        small_df_copy.loc[index, 'content'] = content
    except Exception:
        pass

small_df_copy.to_csv(OUTPUT_CSV, 
                     index=False, quoting=csv.QUOTE_ALL)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f0fda59-c6a8-4e8e-8948-140616c5cf47' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>